In [1]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = "SNP"

# Set the root
from from_root import from_root # type: ignore
ROOT = from_root()

import pandas as pd # type: ignore
from loguru import logger # type: ignore

pd.options.display.max_columns = None
pd.set_option('display.precision', 2)

from pathlib import Path
import sys

# Add `src` and ROOT to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path

if "src" not in Path.cwd().parts:
    src_path = str(Path(get_path("purelib")) / "_src.pth")
    with open(src_path, "w") as f:
        f.write(str(ROOT / "src\n"))
        f.write(str(ROOT))
        if str(ROOT) not in sys.path:
            sys.path.insert(1, str(ROOT))

# Start the Jupyter loop
from ib_async import util # type: ignore

util.startLoop()

In [15]:
df = pd.read_pickle(ROOT/'data'/'pf.pkl')
df.head()

# Group by symbol and get strategy based on secType priority
def get_strategy(group):
    stk_rows = group[group['secType'] == 'STK']
    if not stk_rows.empty:
        return stk_rows['strategy'].iloc[0]
    opt_rows = group[group['secType'] == 'OPT'] 
    if not opt_rows.empty:
        return opt_rows['strategy'].iloc[0]
    return 'NA'

summary = df.groupby('symbol').agg({
    'unPnL': 'sum',
    'secType': 'first',
    'position': 'first'
}).sort_values('unPnL')

# Add strategy based on secType priority
summary['strategy'] = df.groupby('symbol').apply(get_strategy, include_groups=False)

summary['position_type'] = summary.apply(lambda x: 
    'long stock' if x['secType'] == 'STK' and x['position'] > 0 
    else 'short stock' if x['secType'] == 'STK' and x['position'] < 0
    else 'NA', axis=1)

# Display results
print("\nP&L, Position Type and Strategy by Symbol:")
print(summary[['unPnL', 'position_type', 'strategy']].to_string(float_format=lambda x: f"${x:,.2f}"))



P&L, Position Type and Strategy by Symbol:
            unPnL position_type     strategy
symbol                                      
BIIB   $-1,798.08    long stock    uncovered
MRNA   $-1,639.19            NA          cwp
TTWO     $-534.31            NA     orphaned
CAG      $-322.28    long stock    uncovered
DLTR     $-145.58    long stock    uncovered
GOOGL     $-18.93   short stock      exposed
GEHC       $18.00   short stock      exposed
DVN        $35.00   short stock      exposed
MO         $49.00    long stock      exposed
CVS        $72.00   short stock      exposed
ADI        $74.00    long stock      exposed
FOXA       $80.81            NA     orphaned
QCOM       $91.00    long stock      exposed
NXPI      $115.00   short stock      exposed
CNC       $141.00   short stock      exposed
CF        $171.18    long stock      exposed
KHC       $172.00   short stock      exposed
VZ        $200.00   short stock      exposed
NKE       $220.00   short stock      exposed
CMCSA     $

In [16]:
df[df.symbol == 'LLY']

,secType,conId,symbol,expiry,strike,right,contract,position,mktPrice,mktVal,avgCost,unPnL,rePnL,und_price,und_iv,und_hv,strategy
55,STK,9160,LLY,,0.0,0,"Stock(conId=9160, symbol='LLY', right='0', pri...",-100.0,832.48,-83248.0,850.0,1752.0,0.0,797.45,0.29,0.48,exposed
